In [ ]:
from ldap3 import Server, Connection, ALL, IP_V4_ONLY
import getpass
import datetime
import pytz
import humanize
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import serviceping
import socket
def find_fastest_global_catalog():
    sockaddrs = [
        addrinfo[4]
        for addrinfo
        in socket.getaddrinfo('corpad.adbkng.com', 3269, family=socket.AF_INET)]
    ping_results = {
        sockaddr: serviceping.network.ping(sockaddr[0], sockaddr[1], timeout=1.0)
        for sockaddr
        in sockaddrs}
    ping_results = {
        sockaddr: response.durations['all']
        for sockaddr, response
        in ping_results.items()
        if response.responding}
    fastest_server = sorted(ping_results.items(), key=lambda x: x[1])[0][0]
    return fastest_server
fastest_global_catalog = find_fastest_global_catalog()

In [ ]:
login_name = input('Login Username (e.g. "jdoe"): ')
username = 'CORPAD\\' + login_name
password = getpass.getpass('Password: ')

In [ ]:
import os
import shelve
SHELF_FILE = 'ad_persons.shelve'

def ensure_shelf_file():
    if os.path.exists(SHELF_FILE + '.db'):
        print(f"Cache file {SHELF_FILE} already exists. Reading...")
        return None
    server = Server(
        fastest_global_catalog[0],
        port=fastest_global_catalog[1],
        use_ssl=True,
        get_info=ALL,
        mode=IP_V4_ONLY)
    connection = Connection(server, username, password, auto_bind=True)
    begin_search = datetime.datetime.now(tz=pytz.utc)
    print("Downloading entries from Active Directory...")
    entries = connection.extend.standard.paged_search(
        'OU=Exchange Users,OU=Main,DC=corpad,DC=adbkng,DC=com',
        '(objectClass=person)',
        attributes=[
            'title',
            'department',
            'extensionAttribute11', # Join Date
            'extensionAttribute6',  # Job Key
            'userAccountControl',
            'whenChanged',
            'lastLogonTimestamp',
            'sAMAccountName'])
    results = []
    for entry in entries:
        new_attributes = {}
        for attribute, value in entry['attributes'].items():
            if isinstance(value, list):
                if len(value) == 0:
                    new_attributes[attribute] = None
                else:
                    new_attributes[attribute] = value[0]
            else:
                new_attributes[attribute] = value
        entry['attributes'] = new_attributes
        results.append(entry)
    end_search = datetime.datetime.now(tz=pytz.utc)
    print(f"Downloaded {len(results)} entries from AD GC in {humanize.naturaldelta(end_search-begin_search)}")
    with shelve.open(SHELF_FILE) as shelf:
        shelf['results'] = results

def get_dataframe():
    ensure_shelf_file()
    with shelve.open(SHELF_FILE) as shelf:
        results = shelf['results']
    df = pd.DataFrame([r['attributes'] for r in results])
    df = df[df['userAccountControl'].notnull()]
    df = df.astype({'userAccountControl': 'int32'})
    df['account_disabled'] = df['userAccountControl'].apply(lambda row: row & 0x02 == 0x02)
    df['extensionAttribute11'] = df['extensionAttribute11'].astype('datetime64[ns]')
    df = df.rename({
        'extensionAttribute6': 'job_key',
        'extensionAttribute11': 'join_date'},
        axis='columns')
    df['today'] = datetime.date.today()
    df['today'] = df['today'].astype('datetime64')
    return df

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = get_dataframe()
active_accounts = df.query('account_disabled == False')
active_accounts['account_age'] = active_accounts['today'] - active_accounts['join_date']

In [ ]:
# This plots your seniority against a distribution of active account seniorities.
# TODO:
# - Clutser / filter by job keys or titles

seconds_per_year = 86400 * 365
(active_accounts['account_age'].astype('timedelta64[s]') / seconds_per_year).plot.kde()
my_account_age_years = active_accounts[active_accounts['sAMAccountName'] == login_name]['account_age'].astype('timedelta64[s]').iloc[0] / seconds_per_year
#active_accounts.query(f'sAMAccountName == "{login_name}"')
plt.axvline(my_account_age_years, color='red')
plt.text(my_account_age_years + 0.1, 0, login_name)
plt.show()